# 3点リーダー[...]の動作確認
* 以降のRNNの実装において、3点リーダー[...]を利用するので、ここでその動作を確認しておく

In [6]:
import numpy as np

## 3点リーダー[...]の基本的な挙動

In [7]:
# 変数a=[1,2,3]と変数b=[4,5,6]があるとする。
a=np.array([1,2,3])
b=np.array([4,5,6])
print(id(a))
print(id(b))
# 変数aにbの値を代入したい場合、a=bとすると、参照渡しになる。
# 変数が指し示すメモリ位置が同じになる=idが同じになる=実態は1つだけ。
a=b
print(a,b)
print(id(a))
print(id(b))
# 参照渡しなので、bの値を更新すると、aの値も更新されてしまう。
b[0] = 7
print(a,b)

4569316528
4569389296
[4 5 6] [4 5 6]
4569389296
4569389296
[7 5 6] [7 5 6]


In [8]:
# 変数a=[1,2,3]と変数b=[4,5,6]があるとする。
a=np.array([1,2,3])
b=np.array([4,5,6])
print(id(a))
print(id(b))
# 変数aにbの値を代入したい場合、a[...]=bとすると、aのメモリ位置に値が代入される(深いコピーという)。
# 変数が指し示すメモリ位置は元のまま=idは2つのまま=実態は２のまま。
a[...]=b
print(a,b)
print(id(a))
print(id(b))
# 参照渡しではないので、bの値を更新しても、aの値は更新されない。
b[0] = 7
print(a,b)

4569390976
4569390096
[4 5 6] [4 5 6]
4569390976
4569390096
[4 5 6] [7 5 6]


## クラスを継承する場合における3点リーダー[...]の挙動

### 3点リーダー[...]を使わない場合

In [9]:
class A():
    def __init__(self):
        self.a_layers = [B(), B()]
        self.grads = []
        for layer in self.a_layers:
            self.grads += layer.grads

class B():
    def __init__(self):
        self.b_layers = [C(), C()]
        self.grads = []
        for layer in self.b_layers:
            self.grads += layer.grads
        print("id of self.b_layers[0].grads[2]  at init B =", id(self.b_layers[0].grads[2]))
            
class C():
    def __init__(self):
        self.grads = [np.zeros((3,3)), np.zeros((3,3)), np.zeros((3,3))]
        print("id of self.grads[2] at init C =", id(self.grads[2]))
        
    def run(self):
        self.grads[2] = np.ones((3,3)) * 3
        print("id of self.grads[2] at run function=", id(self.grads[2]))
        
a = A()
print(a.grads)

print()
print("*"*100)
print("class Cのrun関数を実行し、gradに代入する")
print("*"*100)

a.a_layers[0].b_layers[0].run()
print("id of a.a_layers[0].b_layers[0].grads[2]=", id(a.a_layers[0].b_layers[0].grads[2]))
print(a.a_layers[0].b_layers[0].grads)
print(a.grads)
print("class Cのgradsがa.gradsに反映されていない")

id of self.grads[2] at init C = 4569575952
id of self.grads[2] at init C = 4569576032
id of self.b_layers[0].grads[2]  at init B = 4569575952
id of self.grads[2] at init C = 4569577472
id of self.grads[2] at init C = 4569577872
id of self.b_layers[0].grads[2]  at init B = 4569577472
[array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), 

### 3点リーダー[...]を使う場合

In [10]:
class A():
    def __init__(self):
        self.a_layers = [B(), B()]
        self.grads = []
        for layer in self.a_layers:
            self.grads += layer.grads

class B():
    def __init__(self):
        self.b_layers = [C(), C()]
        self.grads = []
        for layer in self.b_layers:
            self.grads += layer.grads
        print("id of self.b_layers[0].grads[2]  at init B =", id(self.b_layers[0].grads[2]))
            
class C():
    def __init__(self):
        self.grads = [np.zeros((3,3)), np.zeros((3,3)), np.zeros((3,3))]
        print("id of self.grads[2] at init C =", id(self.grads[2]))
        
    def run(self):
        self.grads[2][...] = np.ones((3,3)) * 3
        print("id of self.grads[2] at run function=", id(self.grads[2]))
        
a = A()
print(a.grads)

print()
print("*"*100)
print("class Cのrun関数を実行し、gradに代入する")
print("*"*100)

a.a_layers[0].b_layers[0].run()
print("id of a.a_layers[0].b_layers[0].grads[2]=", id(a.a_layers[0].b_layers[0].grads[2]))
print(a.a_layers[0].b_layers[0].grads)
print(a.grads)
print("class Cのgradsがa.gradsに反映されている")

id of self.grads[2] at init C = 4569608400
id of self.grads[2] at init C = 4569608560
id of self.b_layers[0].grads[2]  at init B = 4569608400
id of self.grads[2] at init C = 4569608800
id of self.grads[2] at init C = 4550708320
id of self.b_layers[0].grads[2]  at init B = 4569608800
[array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), 

### おな、以下のような書き方をすれば、3点リーダー[...]を使わなくても済むが、汎用性がなくなってしまう
```
for layer in self.b_layers:
    self.grads += layer.grads
```
の部分を
```
self.grads += [self.a_layers[0].grads]
self.grads += [self.a_layers[1].grads]
```
のように変更してあげる

In [11]:
class A():
    def __init__(self):
        self.a_layers = [B(), B()]
        self.grads = []
        self.grads += [self.a_layers[0].grads]
        self.grads += [self.a_layers[1].grads]

class B():
    def __init__(self):
        self.b_layers = [C(), C()]
        self.grads = []
        self.grads += [self.b_layers[0].grads]
        self.grads += [self.b_layers[1].grads]
        print("id of self.b_layers[0].grads[2]  at init B =", id(self.b_layers[0].grads[2]))
            
class C():
    def __init__(self):
        self.grads = [np.zeros((3,3)), np.zeros((3,3)), np.zeros((3,3))]
        print("id of self.grads[2] at init C =", id(self.grads[2]))
        
    def run(self):
        self.grads[2] = np.ones((3,3)) * 3
        print("id of self.grads[2] at run function=", id(self.grads[2]))
        
        
a = A()
print(a.grads)

print()
print("*"*100)
print("class Cのrun関数を実行し、gradに代入する")
print("*"*100)

a.a_layers[0].b_layers[0].run()
print("id of a.a_layers[0].b_layers[0].grads[2]=", id(a.a_layers[0].b_layers[0].grads[2]))
print(a.a_layers[0].b_layers[0].grads)
print(a.grads)
print("class Cのgradsがa.gradsに反映されている")

id of self.grads[2] at init C = 4569317248
id of self.grads[2] at init C = 4569608960
id of self.b_layers[0].grads[2]  at init B = 4569317248
id of self.grads[2] at init C = 4569609120
id of self.grads[2] at init C = 4569609360
id of self.b_layers[0].grads[2]  at init B = 4569609120
[[[array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])], [array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])]], [[array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])], [array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0